# This is Version 1 of our visualization tool
I use the following code to display feature lines (detected by CrestCODE based on curvature)

In [18]:
import os
import numpy as np
import open3d as o3d
import igl
import meshplot as mp

In [82]:
MESHNAME = "tbk_crop_230328_depth=12_correctNormals_crop_1"
MESHFILE = "../Open3D_Test/" + MESHNAME + ".ply"
PSEUDO_PLY2 = 'ply2.txt'
NEWLINE = '\n'
NEIGHBOR = 6
CRESTLINE = 1 # 0 means NO Crest line, 1 means YES Crest line
TXT = ".txt"
RIDGES = "Ridge_" + MESHNAME + TXT
RAVINES = "Ravine_" + MESHNAME + TXT
OUTPUT_rename = "Output_" + MESHNAME + TXT
PLY2_rename = "PLY2_" + MESHNAME + TXT
PLY = ".ply"

In [42]:
# read in mesh file
def get_mesh(path):
    mesh = o3d.io.read_triangle_mesh(MESHFILE)
    print(mesh)
    verts = np.asarray(mesh.vertices)
    faces = np.asarray(mesh.triangles)
    return verts, faces

# turn meshfile to Pseudo_PLY2 file
def to_pseudo_PLY2(name, neighbor, crestline, verts, faces):
    NEWLINE = '\n'
    verts_count = len(verts)
    faces_count = len(faces)
    with open(name, 'w') as f:
        # num vertices
        f.write(str(verts_count) + NEWLINE)
        # num triangles
        f.write(str(faces_count) + NEWLINE)
        # neighborhood size
        f.write(str(neighbor) + NEWLINE)
        # Crestline Y/N
        f.write(str(crestline) + NEWLINE)
        # vertices (coordinates)
        for i in range(verts_count):
            f.write(" ".join(map(str, verts[i])) + NEWLINE)
        # faces (vertex IDs)
        for j in range(faces_count):
            f.write(" ".join(map(str, faces[j])) + NEWLINE)

# get data from Ridges.txt OR Ravines.txt
def read_RnR(FILE):
    f = open(FILE, 'r')
    # num of vertices
    V = int(f.readline())
    # num of edges
    E = int(f.readline())
    # num of connected components
    N = int(f.readline())
    # vertices
    """[x, y, z, connected cmp ID]; index = vtx ID"""
    vertices = np.zeros(shape = (V, 4))
    for i in range(V): 
        #index = pointID
        line = f.readline()
        vertices[i] = [float(n) for n in line.split()]
    """print("first vtx")
    print(vertices[0])
    print("last vtx")
    print(vertices[V - 1])
    print(vertices.dtype)"""
    # connected components
    """[Ridgeness, Sphericalness, Cyclideness]; index = connected cmp ID"""
    connected_cmp = np.zeros(shape = (N, 3))
    for j in range(N):
        line = f.readline()
        connected_cmp[j] = [float(n) for n in line.split()]
    """print("first connected cmp")
    print(connected_cmp[0])
    print("last connected cmp")
    print(connected_cmp[N - 1])
    print(connected_cmp.dtype)"""
    # edges (u,v)
    """[vtx ID of u, vtx ID of v, triangle ID]; index = edge ID"""
    edges = np.zeros(shape = (E, 3), dtype=int)
    for k in range(E):
        line = f.readline()
        edges[k] = [n for n in line.split()]
    """print("first edge")
    print(edges[0])
    print("last edge:")
    print(edges[E - 1])
    print(edges.dtype)"""
    return (E, vertices, edges)

# get vtx pair for Crest Line edges!
def preprop_RnR(tuple):
    S = np.zeros(shape = (tuple[0], 3))
    T = np.zeros(shape = (tuple[0], 3))
    for i in range(tuple[0]):
        S[i] = tuple[1][tuple[2][i, 0]][0:3]
        T[i] = tuple[1][tuple[2][i, 1]][0:3]
    return S, T

def show(ridges, ravines, verts, faces):
    p = mp.plot(verts, faces, return_plot=True)
    RI_S, RI_T = preprop_RnR(read_RnR(ridges))
    p.add_lines(RI_S, RI_T, shading={"line_color": "red", "line_width": 10.0})
    RA_S, RA_T = preprop_RnR(read_RnR(ravines))
    p.add_lines(RA_S, RA_T, shading={"line_color": "blue", "line_width": 10.0})
    p.save("Ridge&Ravine_" + MESHNAME)

def ridges(ridges, verts, faces):
    p = mp.plot(verts, faces, return_plot=True)
    RI_S, RI_T = preprop_RnR(read_RnR(ridges))
    p.add_lines(RI_S, RI_T, shading={"line_color": "red", "line_width": 10.0})
    p.save("Ridge_" + MESHNAME)

def ravines(ravines, verts, faces):
    p = mp.plot(verts, faces, return_plot=True)
    RA_S, RA_T = preprop_RnR(read_RnR(ravines))
    p.add_lines(RA_S, RA_T, shading={"line_color": "blue", "line_width": 10.0})
    p.save("Ravine_" + MESHNAME)

In [43]:
verts, faces = get_mesh(MESHFILE)

TriangleMesh with 183375 points and 364868 triangles.


In [44]:
to_pseudo_PLY2(PSEUDO_PLY2, NEIGHBOR, CRESTLINE, verts, faces)

# BOOM! You ran CrestCODE and got your .txt outputs!
<br>You ran command in the CrestCODE folder (through Terminal) and got .txt files for: output, ravines, ridges
>the input: ./setCurvature ply2.txt output.txt
><br>Note that 'ply2.txt' corresponds to (the value of) PSEUDO_PLY2

<br>The .txt show up in the CrestCODE folder
<br>You moved those files Back to Open3D_Test!

In [45]:
#cleaning up!
os.rename("ridges.txt", RIDGES)
os.rename("ravines.txt", RAVINES)
os.rename("output.txt", OUTPUT_rename)
os.rename("ply2.txt", PLY2_rename)

In [ ]:
# all feature lines
show(RIDGES, RAVINES, verts, faces)

In [ ]:
# just show ridges (not very useful)
ridges(RIDGES, verts, faces)

In [ ]:
# show ravines (very nice)
ravines(RAVINES, verts, faces)

In [ ]:
# show entire mesh (no nothing)
p = mp.plot(verts, faces, return_plot=True)

In [52]:
# Save
p.save("FeatureLines_" + MESHNAME + "_neighbor=" + str(NEIGHBOR))

Plot saved to file FeatureLines_tbk_crop_230328_depth=12_correctNormals_crop_2_neighbor=6.html.


In [83]:
# Reference: 
# http://www.open3d.org/docs/0.11.1/tutorial/visualization/visualization.html
(edge_ct, edge_vertices, edge_indices) = read_RnR(RAVINES)
points = edge_vertices[:,:3]
lines = edge_indices[:,:2]
colors = [[1, 0, 0] for i in range(edge_ct)]
line_set = o3d.geometry.LineSet(
    points=o3d.utility.Vector3dVector(points),
    lines=o3d.utility.Vector2iVector(lines),
)
line_set.colors = o3d.utility.Vector3dVector(colors)
o3d.io.write_line_set(RAVINES + PLY, line_set, print_progress=True)

Writing PLY: [========================================] 100%


True

In [62]:
# make empty lineSet
lines = o3d.geometry.LineSet()
# position of vertices
PointPositions = o3d.core.Tensor(edge_vertices, o3d.core.float32)
# indices of each edge: (u_index, v_index)
LineIndices = o3d.core.Tensor(edge_indices, o3d.core.int32)
# assign these to LineSet
lines.points = PointPositions
lines.lines = LineIndices
succeed = o3d.io.write_line_set("lines.ply", lines, print_progress=True)

TypeError: (): incompatible function arguments. The following argument types are supported:
    1. (self: open3d.cpu.pybind.geometry.LineSet, arg0: open3d.cpu.pybind.utility.Vector3dVector) -> None

Invoked with: LineSet with 0 lines., [[1.24957 1.86273 -0.21515 0.0],
 [1.24957 1.8627 -0.215149 0.0],
 [1.2498 1.86506 -0.215192 0.0],
 [1.24979 1.8644 -0.215192 0.0],
 [1.25018 1.86567 -0.215218 0.0],
 [1.25030 1.86595 -0.215224 0.0],
 [1.25034 1.86607 -0.215226 0.0],
 [1.25185 1.86753 -0.215278 0.0],
 [1.25199 1.8676 -0.21528 0.0],
 [1.25199 1.86774 -0.215281 0.0],
 [1.25994 1.86273 -0.215326 1.0],
 [1.26009 1.86285 -0.215333 1.0],
 [1.25961 1.8644 -0.215366 1.0],
 [1.25853 1.86522 -0.215325 1.0],
 [1.26187 1.85192 -0.215181 2.0],
 [1.26178 1.85104 -0.215155 2.0],
 [1.26054 1.85118 -0.215149 3.0],
 [1.26025 1.85258 -0.215179 3.0],
 [1.26246 1.85271 -0.215187 2.0],
 [1.262 1.85267 -0.2152 2.0],
 [1.26224 1.85162 -0.215168 2.0],
 [1.14658 1.91826 -0.215158 4.0],
 [1.14638 1.91898 -0.215149